In [ ]:
#import necessary packeges
from keras.models import Sequential
from keras.layers import Conv2D, Input
from keras.optimizers import SGD, Adam
#from skimage.measure import compare_ssim as ssim
from skimage.metrics import structural_similarity as ssim
from matplotlib import pyplot as plt
import cv2
import numpy as np
import math
import os

#Import magic funtion
%matplotlib inline

print('Done')

In [ ]:
#Define a function for peak signal-to-noise ration (PSNR)
def psnr(target, ref):
    #Define RGB/BGR image
    target_data = target.astype(float)
    ref_data = ref.astype(float)
    
    diff = ref_data - target_data
    diff = diff.flatten('C')
    
    rmse = math.sqrt(np.mean(diff**2.))
    return 20 * math.log10(255. / rmse)

#Define a function of mean square error
def mse(target,ref):
    #MSE sum of squared difference between two images
    err = np.sum((target.astype('float') - ref.astype('float')) ** 2)
    err /= float(target.shape[0]*target.shape[1])
    return err

#Define function that combine all three image quality matics
def compare_image(target, ref):
    scores = []
    scores.append(psnr(target, ref))
    scores.append(mse(target, ref))
    scores.append(ssim(target, ref, multichannel = True))
    return scores

Image Source
http://mmlab.ie.cuhk.edu.hk/projects/SRCNN.html

In [ ]:
# prepare degraded images by introducing quality distortions via resizing

def prepare_images(path, factor):
    
    # loop through the files in the directory
    for file in os.listdir(path):
        
        # open the file
        img = cv2.imread(path + '/' + file)
        
        # find old and new image dimensions
        h, w, _ = img.shape
        new_height = int(h / factor)
        new_width = int(w / factor)
        
        # resize the image - down
        img = cv2.resize(img, (new_width, new_height), interpolation = cv2.INTER_LINEAR)
        
        # resize the image - up
        img = cv2.resize(img, (w, h), interpolation = cv2.INTER_LINEAR)
        
        # save the image
        print('Saving {}'.format(file))
        cv2.imwrite('images/{}'.format(file), img)

In [ ]:
prepare_images('source/',2)

In [ ]:
for file in os.listdir('images/'):
    #open tager and references images
    target = cv2.imread('images/{}'.format(file))
    ref = cv2.imread('source/{}'.format(file))
    
    #Calculate the scoure
    scores = compare_image(target,ref)
    
    #Print all three sources
    print('{}\nPSNR: {}\nSSIN: {}'.format(file, scores[0], scores[1], scores[2]))

In [ ]:
#Define the SRCNN model
def model():
    #Define model type
    SRCNN = Sequential()
    
    #Add model layer
    SRCNN.add(Conv2D(filters=128, kernel_size=(9,9), kernel_initializer='glorot_uniform',
                    activation = 'relu', padding='valid', use_bias=True, input_shape=(None, None, 1)))
    SRCNN.add(Conv2D(filters=64, kernel_size=(3,3), kernel_initializer='glorot_uniform',
                    activation = 'relu', padding='same', use_bias=True))
    SRCNN.add(Conv2D(filters=1, kernel_size=(5,5), kernel_initializer='glorot_uniform',
                    activation='linear', padding='valid', use_bias=True))
    
    #Define Optmizer
    adam = Adam(lr=0.0003)
    
    #Compile model
    SRCNN.compile(optimizer=adam, loss = 'mean_squared_error', metrics = ['mean_squared_error'])
    
    return SRCNN

GitHub page: https://github.com/MarkPrecursor/SRCNN-keras

In [ ]:
#Define necessary image processing function
def modcrop(img, scale):
    tmpsz = img.shape
    sz = tmpsz[0:2]
    sz = sz - np.mod(sz, scale)
    img = img[0:sz[0], 1:sz[1]]
    return img

def shave(image, border):
    img = image[border: -border, border: -border]
    return img

In [ ]:
#Define main prediction function
def predict(image_path):
    #load the srcnn model
    srcnn = model()
    srcnn.load_weights('3051crop_weight_200.h5')
    
    
    #Load the degreded and reference images
    path, file = os.path.split(image_path)
    degraded = cv2.imread(image_path)
    ref = cv2.imread('source/{}'.format(file))
    
    #Preproces the images with modcrop
    ref = modcrop(ref,3)
    degraded = modcrop(degraded, 3)
    
    #Conver the images YCrCb - (srcnn training on Y chanell)
    temp = cv2.cvtColor(degraded, cv2.COLOR_BGR2YCrCb)
    
    #Create images slice and normalize
    Y = np.zeros((1, temp.shape[0], temp.shape[1], 1), dtype=float)
    Y[0,:,:,0] = temp[:,:,0].astype(float)/255
    
    #Perform super-resolution  with srcnn
    pre = srcnn.predict(Y, batch_size = 1)
    
    #Post Process output
    pre*= 255
    pre[pre[:] > 255] = 255
    pre[pre[:] < 0] = 0
    pre = pre.astype(np.uint8)
    
    #Copy the y chanell back to the images and convert to RGB
    temp = shave(temp,6)
    temp[:,:,0] = pre[0,:,:,0]
    output = cv2.cvtColor(temp, cv2.COLOR_YCrCb2BGR)
    
    #Remove the border from the reference and degreaded images
    ref = shave(ref.astype(np.uint8), 6)
    degraded = shave(degraded.astype(np.uint8), 6)
    
    #Images Quality Calculation
    scores = []
    scores.append(compare_image(degraded,ref))
    scores.append(compare_image(output,ref))
    
    #Return images and scores
    return ref, degraded, output, scores

In [ ]:
ref, degraded, output, scores = predict('images/flowers.bmp')

#print scores for all images
print('Degraded Image: \nPSNR: {}\nMSE: {}\nSSIM: {}'.format(scores[0][0], scores[0][1], scores[0][2]))
print('Reconstructed Image: \nPSNR {}\nMSE: {}\nSSIM: {}'.format(scores[1][0], scores[1][1], scores[1][2]))

#Dispaly Images as subplot
fig, axs = plt.subplots(1,3,figsize = (20,8))
axs[0].imshow(cv2.cvtColor(ref, cv2.COLOR_BGR2RGB))
axs[0].set_title('Orginal')

axs[1].imshow(cv2.cvtColor(degraded, cv2.COLOR_BGR2RGB))
axs[1].set_title('Degraded')

axs[2].imshow(cv2.cvtColor(output, cv2.COLOR_BGR2RGB))
axs[2].set_title('SRCNN')

#Remove x and y tick marks
for ax in axs:
    ax.set_xticks([])
    ax.set_yticks([])

In [ ]:
for file in os.listdir('images'):
    ref, degraded, output, scores = predict('images/flowers.bmp')

    #print scores for all images
    print('Degraded Image: \nPSNR: {}\nMSE: {}\nSSIM: {}'.format(scores[0][0], scores[0][1], scores[0][2]))
    print('Reconstructed Image: \nPSNR {}\nMSE: {}\nSSIM: {}'.format(scores[1][0], scores[1][1], scores[1][2]))

    #Dispaly Images as subplot
    fig, axs = plt.subplots(1,3,figsize = (20,8))
    axs[0].imshow(cv2.cvtColor(ref, cv2.COLOR_BGR2RGB))
    axs[0].set_title('Orginal')

    axs[1].imshow(cv2.cvtColor(degraded, cv2.COLOR_BGR2RGB))
    axs[1].set_title('Degraded')
    axs[1].set(xlabel = 'PSNR: {}\nMSE: {} \nSSIM: {}'.format(scores[0][0], scores[0][1], scores[0][2]))

    axs[2].imshow(cv2.cvtColor(output, cv2.COLOR_BGR2RGB))
    axs[2].set_title('SRCNN')
    axs[2].set(xlabel = 'PSNR: {} \nMSE: {} \nSSIM: {}'.format(scores[1][0], scores[1][1], scores[1][2]))

    #Remove x and y tick marks
    for ax in axs:
        ax.set_xticks([])
        ax.set_yticks([])
    
    #Save the images
    print('Saving {}'.format(file))
    fig.savefig('output/{}.png'.format(os.path.splitext(file)[0]))
    plt.close()
        